In [2]:
# 1
import pandas as pd
import os
import plotly.graph_objects as go

In [5]:
ruta_de_archivos = r'C:\Users\rafae\bigData\Dataset de ventas'

archivos = os.listdir(ruta_de_archivos)

dataframes = {}

for archivo in archivos:
    mes = archivo.split('_')[3].split('.')[0]
    ruta_completa = os.path.join(ruta_de_archivos, archivo)
    df = pd.read_csv(ruta_completa)
    df_ventas = df.copy()

#   filtrado y procesamiento de datos
    df_ventas['Cantidad Pedida'] = pd.to_numeric(df_ventas['Cantidad Pedida'], errors='coerce')
    df_ventas['Precio Unitario'] = pd.to_numeric(df_ventas['Precio Unitario'], errors='coerce')
    df_ventas['ID de Pedido'] = pd.to_numeric(df_ventas['ID de Pedido'], errors='coerce')


    # eliminar filas
    df_ventas = df_ventas[~df_ventas['Cantidad Pedida'].isna()]
    df_ventas = df_ventas[~df_ventas['Precio Unitario'].isna()]
    df_ventas = df_ventas[~df_ventas['ID de Pedido'].isna()]

    # acomodar columnas
    df_ventas['Cantidad Pedida'] = df_ventas['Cantidad Pedida'].astype(int)
    df_ventas['Precio Unitario'] = df_ventas['Precio Unitario'].astype(int)
    df_ventas['ID de Pedido'] = df_ventas['ID de Pedido'].astype(int)

    dataframes[mes] = df_ventas

meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

dataframes_mes = {}


In [6]:
# pregunta 1 
# ¿Cómo variaron las ventas a lo largo de los diferentes meses?

totales_ventas = []

for mes in meses:
    dataframes_mes = dataframes[mes]
    totales_ventas_mes = dataframes_mes['Cantidad Pedida'].sum()
    totales_ventas.append(totales_ventas_mes)

        # Crear columna de ingresos totales
    dataframes_mes['Ingreso Total'] = dataframes_mes['Cantidad Pedida'] * dataframes_mes['Precio Unitario']
    
    # Sumar el total de ingresos
    ingreso_total_mes = dataframes_mes['Ingreso Total'].sum()
    print(f"Ingreso total para el mes {mes}: ${ingreso_total_mes}")

totales_ventas

# grafico
fig = go.Figure()

fig.add_trace(go.Bar(
    x = meses,
    y = totales_ventas,
    marker = dict(color=totales_ventas,colorscale= 'RdYlGn')
))

fig.show()

Ingreso total para el mes Enero: $1807060
Ingreso total para el mes Febrero: $2193144
Ingreso total para el mes Marzo: $2796031
Ingreso total para el mes Abril: $3380338
Ingreso total para el mes Mayo: $3130267
Ingreso total para el mes Junio: $2566675
Ingreso total para el mes Julio: $2634571
Ingreso total para el mes Agosto: $2234143
Ingreso total para el mes Septiembre: $2088683
Ingreso total para el mes Octubre: $3719364
Ingreso total para el mes Noviembre: $3183687
Ingreso total para el mes Diciembre: $4597677


In [35]:
#2
import pandas as pd
import glob
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

# Ruta donde se encuentran los archivos CSV
path = r"C:\Users\rafae\bigData\Dataset de ventas\\"

# Cargar todos los archivos CSV que empiezan con "Ventas_" en un solo DataFrame
all_files = glob.glob(path + "Dataset_de_ventas_*.csv")

# Cargar y concatenar los archivos CSV
df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

# Convertir la columna 'Fecha' a formato datetime y extraer 'Hora' y 'Mes'
# df['Fecha de Pedido'] = pd.to_datetime(df['Fecha de Pedido'])
# df['Fecha de Pedido'] = df['Fecha de Pedido'].dt.hour
# df['mes'] = df['Fecha de Pedido'].dt.month

# Eliminar filas donde la columna 'Fecha de Pedido' tenga valores no válidos
df = df[df['Fecha de Pedido'] != 'Order Date']

# Intentar convertir la columna 'Fecha de Pedido' a formato datetime con el formato adecuado
df['Fecha de Pedido'] = pd.to_datetime(df['Fecha de Pedido'], format='%m/%d/%y %H:%M', errors='coerce')

# Eliminar las filas que no se pudieron convertir a datetime
df = df[df['Fecha de Pedido'].notna()]

# Continuar con la extracción de 'Hora' y 'Mes'
df['Hora'] = df['Fecha de Pedido'].dt.hour
df['Mes'] = df['Fecha de Pedido'].dt.month


# 1. Análisis de correlación (Pearson) entre ventas y horas del día

# Contar el número de ventas por cada hora del día
ventas_por_hora = df.groupby('Hora').size()

# Normalizar las ventas para asegurar consistencia
ventas_normalizadas = (ventas_por_hora - np.mean(ventas_por_hora)) / np.std(ventas_por_hora)

# Calcular la correlación de Pearson entre horas y ventas normalizadas
horas = ventas_por_hora.index
correlacion, p_valor = pearsonr(horas, ventas_normalizadas)

print(f"Correlación de Pearson entre horas y ventas: {correlacion}")
print(f"Valor p: {p_valor}")

Correlación de Pearson entre horas y ventas: 0.7479323629351873
Valor p: 2.6489753790346927e-05


In [7]:
# Filtrar ventas durante las horas pico
horas_pico = df[df['Hora'].isin([18, 19, 20, 21])]

# Agrupar por mes para ver las ventas en horas pico
ventas_pico_mes = horas_pico.groupby('Mes').size()

# Graficar las ventas en horas pico por mes
plt.figure(figsize=(10,6))
sns.barplot(x=ventas_pico_mes.index, y=ventas_pico_mes.values, palette="coolwarm")
plt.title('Ventas Durante Horas Pico por Mes')
plt.xlabel('Mes')
plt.ylabel('Número de Ventas')
plt.show()


KeyError: 'Hora'